# Variational autoencoders for collaborative filtering 

This notebook accompanies the paper "*Variational autoencoders for collaborative filtering*" by Dawen Liang, Rahul G. Krishnan, Matthew D. Hoffman, and Tony Jebara, in The Web Conference (aka WWW) 2018.

In this notebook, we will show a complete self-contained example of training a variational autoencoder (as well as a denoising autoencoder) with multinomial likelihood (described in the paper) on the public Movielens-20M dataset, including both data preprocessing and model training.

In [1]:
import os
import shutil
import sys
import math

import numpy as np
from scipy import sparse

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sn
sn.set()

import pandas as pd

import tensorflow as tf
from tensorflow.contrib.layers import apply_regularization, l2_regularizer

import bottleneck as bn

/home/tonylab/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tonylab/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tonylab/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/tonylab/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
### change `DATA_DIR` to the location where movielens-20m dataset sits
DATA_DIR = './'
pro_dir = os.path.join(DATA_DIR, 'pro_sg')

## Data preprocessing

We load the data and create train/validation/test splits following strong generalization: 

- We split all users into training/validation/test sets. 

- We train models using the entire click history of the training users. 

- To evaluate, we take part of the click history from held-out (validation and test) users to learn the necessary user-level representations for the model and then compute metrics by looking at how well the model ranks the rest of the unseen click history from the held-out users.

First, download the dataset at http://files.grouplens.org/datasets/movielens/ml-20m.zip

In [3]:
# raw_data = pd.read_csv(os.path.join('../../User/output/', 'UserFollowingsMat.csv'))
# raw_data

In [4]:
# raw_data.rename(columns={'Unnamed: 0':'userId'}, inplace=True)
# raw_data = raw_data.set_index('userId')
# raw_data

In [5]:
# df = pd.DataFrame(columns=['userId', 'movieId'])
# for uid, row in raw_data.iterrows():
#     for movie in row[row == 1].index:
#         df = df.append({'userId': uid, 'movieId': movie}, ignore_index = True)

# df

In [6]:
# df.to_csv('../UserFollowingRecord.csv', index = 0)

In [7]:
# df = pd.read_csv('../UserFollowingRecord.csv')
# raw_data = df
# raw_data

### Data splitting procedure

- Select 10K users as heldout users, 10K users as validation users, and the rest of the users for training
- Use all the items from the training users as item set
- For each of both validation and test user, subsample 80% as fold-in data and the rest for prediction 

In [8]:
# def get_count(tp, id):
#     playcount_groupbyid = tp[[id]].groupby(id, as_index=False)
#     count = playcount_groupbyid.size()
#     return count

In [9]:
# def filter_triplets(tp, min_uc=5, min_sc=0):
#     # Only keep the triplets for items which were clicked on by at least min_sc users. 
#     if min_sc > 0:
#         itemcount = get_count(tp, 'movieId')
#         tp = tp[tp['movieId'].isin(itemcount.index[itemcount >= min_sc])]
    
#     # Only keep the triplets for users who clicked on at least min_uc items
#     # After doing this, some of the items will have less than min_uc users, but should only be a small proportion
#     if min_uc > 0:
#         usercount = get_count(tp, 'userId')
#         tp = tp[tp['userId'].isin(usercount.index[usercount >= min_uc])]
    
#     # Update both usercount and itemcount after filtering
#     usercount, itemcount = get_count(tp, 'userId'), get_count(tp, 'movieId') 
#     return tp, usercount, itemcount

Only keep items that are clicked on by at least 5 users

In [10]:
# raw_data, user_activity, item_popularity = filter_triplets(raw_data)

In [11]:
# sparsity = 1. * raw_data.shape[0] / (user_activity.shape[0] * item_popularity.shape[0])

# print("After filtering, there are %d watching events from %d users and %d movies (sparsity: %.3f%%)" % 
#       (raw_data.shape[0], user_activity.shape[0], item_popularity.shape[0], sparsity * 100))

In [12]:
# unique_uid = user_activity.index

# np.random.seed(98765)
# idx_perm = np.random.permutation(unique_uid.size)
# unique_uid = unique_uid[idx_perm]

In [13]:
# unique_uid

In [14]:
# # create train/validation/test users
# n_users = unique_uid.size
# n_heldout_users = 150

# tr_users = unique_uid[:(n_users - n_heldout_users * 2)] # -1282
# vd_users = unique_uid[(n_users - n_heldout_users * 2): (n_users - n_heldout_users)] # 1282-1432
# te_users = unique_uid[(n_users - n_heldout_users):] # 1432- 

In [15]:
# train_plays = raw_data.loc[raw_data['userId'].isin(tr_users)]

In [16]:
# unique_sid = pd.unique(raw_data['movieId']) #train_plays['movieId'])

In [17]:
# show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
# profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))

In [18]:
# if not os.path.exists(pro_dir):
#     os.makedirs(pro_dir)

# with open(os.path.join(pro_dir, 'unique_sid.txt'), 'w') as f:
#     for sid in unique_sid:
#         f.write('%s\n' % sid)

In [19]:
# def split_train_test_proportion(data, test_prop=0.2):
#     data_grouped_by_user = data.groupby('userId')
#     tr_list, te_list = list(), list()

#     np.random.seed(98765)

#     for i, (_, group) in enumerate(data_grouped_by_user):
#         n_items_u = len(group)

#         if n_items_u >= 5:
#             idx = np.zeros(n_items_u, dtype='bool')
#             idx[np.random.choice(n_items_u, size=int(test_prop * n_items_u), replace=False).astype('int64')] = True

#             tr_list.append(group[np.logical_not(idx)])
#             te_list.append(group[idx])
#         else:
#             tr_list.append(group)

#         if i % 100 == 0:
#             print("%d users sampled" % i)
#             sys.stdout.flush()

#     data_tr = pd.concat(tr_list)
#     data_te = pd.concat(te_list)
    
#     return data_tr, data_te

In [20]:
# vad_plays = raw_data.loc[raw_data['userId'].isin(vd_users)]
# vad_plays = vad_plays.loc[vad_plays['movieId'].isin(unique_sid)]

In [21]:
# vad_plays_tr, vad_plays_te = split_train_test_proportion(vad_plays)

In [22]:
# test_plays = raw_data.loc[raw_data['userId'].isin(te_users)]
# test_plays = test_plays.loc[test_plays['movieId'].isin(unique_sid)]

In [23]:
# test_plays_tr, test_plays_te = split_train_test_proportion(test_plays)

### Save the data into (user_index, item_index) format

In [24]:
# def numerize(tp):
#     uid = list(map(lambda x: profile2id[x], tp['userId']))
#     sid = list(map(lambda x: show2id[x], tp['movieId']))
#     return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])

In [25]:
# train_data = numerize(train_plays)
# train_data.to_csv(os.path.join(pro_dir, 'train.csv'), index=False)

In [26]:
# vad_data_tr = numerize(vad_plays_tr)
# vad_data_tr.to_csv(os.path.join(pro_dir, 'validation_tr.csv'), index=False)

In [27]:
# vad_data_te = numerize(vad_plays_te)
# vad_data_te.to_csv(os.path.join(pro_dir, 'validation_te.csv'), index=False)

In [28]:
# test_data_tr = numerize(test_plays_tr)
# test_data_tr.to_csv(os.path.join(pro_dir, 'test_tr.csv'), index=False)

In [29]:
# test_data_te = numerize(test_plays_te)
# test_data_te.to_csv(os.path.join(pro_dir, 'test_te.csv'), index=False)

## Model definition and training

We define two related models: denoising autoencoder with multinomial likelihood (Multi-DAE in the paper) and partially-regularized variational autoencoder with multinomial likelihood (Multi-VAE^{PR} in the paper).

### Model definition

__Notations__: We use $u \in \{1,\dots,U\}$ to index users and $i \in \{1,\dots,I\}$ to index items. In this work, we consider learning with implicit feedback. The user-by-item interaction matrix is the click matrix $\mathbf{X} \in \mathbb{N}^{U\times I}$. The lower case $\mathbf{x}_u =[X_{u1},\dots,X_{uI}]^\top \in \mathbb{N}^I$ is a bag-of-words vector with the number of clicks for each item from user u. We binarize the click matrix. It is straightforward to extend it to general count data.

__Generative process__: For each user $u$, the model starts by sampling a $K$-dimensional latent representation $\mathbf{z}_u$ from a standard Gaussian prior. The latent representation $\mathbf{z}_u$ is transformed via a non-linear function $f_\theta (\cdot) \in \mathbb{R}^I$ to produce a probability distribution over $I$ items $\pi (\mathbf{z}_u)$ from which the click history $\mathbf{x}_u$ is assumed to have been drawn:

$$
\mathbf{z}_u \sim \mathcal{N}(0, \mathbf{I}_K),  \pi(\mathbf{z}_u) \propto \exp\{f_\theta (\mathbf{z}_u\},\\
\mathbf{x}_u \sim \mathrm{Mult}(N_u, \pi(\mathbf{z}_u))
$$

The objective for Multi-DAE for a single user $u$ is:
$$
\mathcal{L}_u(\theta, \phi) = \log p_\theta(\mathbf{x}_u | g_\phi(\mathbf{x}_u))
$$
where $g_\phi(\cdot)$ is the non-linear "encoder" function.

In [30]:
class MultiDAE(object):
    def __init__(self, p_dims, q_dims=None, lam=0.01, lr=1e-3, random_seed=None):
        self.p_dims = p_dims
        if q_dims is None:
            self.q_dims = p_dims[::-1]
        else:
            assert q_dims[0] == p_dims[-1], "Input and output dimension must equal each other for autoencoders."
            assert q_dims[-1] == p_dims[0], "Latent dimension for p- and q-network mismatches."
            self.q_dims = q_dims
        self.dims = self.q_dims + self.p_dims[1:]
        
        self.lam = lam
        self.lr = lr
        self.random_seed = random_seed

        self.construct_placeholders()

    def construct_placeholders(self):        
        self.input_ph = tf.placeholder(
            dtype=tf.float32, shape=[None, self.dims[0]])
        self.keep_prob_ph = tf.placeholder_with_default(1.0, shape=None)

    def build_graph(self):

        self.construct_weights()

        saver, logits = self.forward_pass()
        log_softmax_var = tf.nn.log_softmax(logits)

        # per-user average negative log-likelihood
        neg_ll = -tf.reduce_mean(tf.reduce_sum(
            log_softmax_var * self.input_ph, axis=1))
        # apply regularization to weights
        reg = l2_regularizer(self.lam)
        reg_var = apply_regularization(reg, self.weights)
        # tensorflow l2 regularization multiply 0.5 to the l2 norm
        # multiply 2 so that it is back in the same scale
        loss = neg_ll + 2 * reg_var
        
        train_op = tf.train.AdamOptimizer(self.lr).minimize(loss)

        # add summary statistics
        tf.summary.scalar('negative_multi_ll', neg_ll)
        tf.summary.scalar('loss', loss)
        merged = tf.summary.merge_all()
        return saver, logits, loss, train_op, merged

    def forward_pass(self):
        # construct forward graph        
        h = tf.nn.l2_normalize(self.input_ph, 1)
        h = tf.nn.dropout(h, self.keep_prob_ph)
        
        for i, (w, b) in enumerate(zip(self.weights, self.biases)):
            h = tf.matmul(h, w) + b
            
            if i != len(self.weights) - 1:
                h = tf.nn.tanh(h)
        return tf.train.Saver(), h

    def construct_weights(self):

        self.weights = []
        self.biases = []
        
        # define weights
        for i, (d_in, d_out) in enumerate(zip(self.dims[:-1], self.dims[1:])):
            weight_key = "weight_{}to{}".format(i, i+1)
            bias_key = "bias_{}".format(i+1)
            
            self.weights.append(tf.get_variable(
                name=weight_key, shape=[d_in, d_out],
                initializer=tf.contrib.layers.xavier_initializer(
                    seed=self.random_seed)))
            
            self.biases.append(tf.get_variable(
                name=bias_key, shape=[d_out],
                initializer=tf.truncated_normal_initializer(
                    stddev=0.001, seed=self.random_seed)))
            
            # add summary stats
            tf.summary.histogram(weight_key, self.weights[-1])
            tf.summary.histogram(bias_key, self.biases[-1])

The objective of Multi-VAE^{PR} (evidence lower-bound, or ELBO) for a single user $u$ is:
$$
\mathcal{L}_u(\theta, \phi) = \mathbb{E}_{q_\phi(z_u | x_u)}[\log p_\theta(x_u | z_u)] - \beta \cdot KL(q_\phi(z_u | x_u) \| p(z_u))
$$
where $q_\phi$ is the approximating variational distribution (inference model). $\beta$ is the additional annealing parameter that we control. The objective of the entire dataset is the average over all the users. It can be trained almost the same as Multi-DAE, thanks to reparametrization trick. 

In [31]:
class MultiVAE(MultiDAE):

    def construct_placeholders(self):
        super(MultiVAE, self).construct_placeholders()

        # placeholders with default values when scoring
        self.is_training_ph = tf.placeholder_with_default(0., shape=None)
        self.anneal_ph = tf.placeholder_with_default(1., shape=None)
        
    def build_graph(self):
        self._construct_weights()

        saver, logits, KL = self.forward_pass()
        log_softmax_var = tf.nn.log_softmax(logits)

        neg_ll = -tf.reduce_mean(tf.reduce_sum(
            log_softmax_var * self.input_ph,
            axis=-1))
        # apply regularization to weights
        reg = l2_regularizer(self.lam)
        
        reg_var = apply_regularization(reg, self.weights_q + self.weights_p)
        # tensorflow l2 regularization multiply 0.5 to the l2 norm
        # multiply 2 so that it is back in the same scale
        neg_ELBO = neg_ll + self.anneal_ph * KL + 2 * reg_var
        
        train_op = tf.train.AdamOptimizer(self.lr).minimize(neg_ELBO)

        # add summary statistics
        tf.summary.scalar('negative_multi_ll', neg_ll)
        tf.summary.scalar('KL', KL)
        tf.summary.scalar('neg_ELBO_train', neg_ELBO)
        merged = tf.summary.merge_all()

        return saver, logits, neg_ELBO, train_op, merged
    
    def q_graph(self):
        mu_q, std_q, KL = None, None, None
        
        h = tf.nn.l2_normalize(self.input_ph, 1)
        h = tf.nn.dropout(h, self.keep_prob_ph)
        
        for i, (w, b) in enumerate(zip(self.weights_q, self.biases_q)):
            h = tf.matmul(h, w) + b
            
            if i != len(self.weights_q) - 1:
                h = tf.nn.tanh(h)
            else:
                mu_q = h[:, :self.q_dims[-1]]
                logvar_q = h[:, self.q_dims[-1]:]

                std_q = tf.exp(0.5 * logvar_q)
                KL = tf.reduce_mean(tf.reduce_sum(
                        0.5 * (-logvar_q + tf.exp(logvar_q) + mu_q**2 - 1), axis=1))
        return mu_q, std_q, KL

    def p_graph(self, z):
        h = z
        
        for i, (w, b) in enumerate(zip(self.weights_p, self.biases_p)):
            h = tf.matmul(h, w) + b
            
            if i != len(self.weights_p) - 1:
                h = tf.nn.tanh(h)
        return h

    def forward_pass(self):
        # q-network
        mu_q, std_q, KL = self.q_graph()
        epsilon = tf.random_normal(tf.shape(std_q))

        sampled_z = mu_q + self.is_training_ph *\
            epsilon * std_q

        # p-network
        logits = self.p_graph(sampled_z)
        
        return tf.train.Saver(), logits, KL

    def _construct_weights(self):
        self.weights_q, self.biases_q = [], []
        
        for i, (d_in, d_out) in enumerate(zip(self.q_dims[:-1], self.q_dims[1:])):
            if i == len(self.q_dims[:-1]) - 1:
                # we need two sets of parameters for mean and variance,
                # respectively
                d_out *= 2
            weight_key = "weight_q_{}to{}".format(i, i+1)
            bias_key = "bias_q_{}".format(i+1)
            
            self.weights_q.append(tf.get_variable(
                name=weight_key, shape=[d_in, d_out],
                initializer=tf.contrib.layers.xavier_initializer(
                    seed=self.random_seed)))
            
            self.biases_q.append(tf.get_variable(
                name=bias_key, shape=[d_out],
                initializer=tf.truncated_normal_initializer(
                    stddev=0.001, seed=self.random_seed)))
            
            # add summary stats
            tf.summary.histogram(weight_key, self.weights_q[-1])
            tf.summary.histogram(bias_key, self.biases_q[-1])
            
        self.weights_p, self.biases_p = [], []

        for i, (d_in, d_out) in enumerate(zip(self.p_dims[:-1], self.p_dims[1:])):
            weight_key = "weight_p_{}to{}".format(i, i+1)
            bias_key = "bias_p_{}".format(i+1)
            self.weights_p.append(tf.get_variable(
                name=weight_key, shape=[d_in, d_out],
                initializer=tf.contrib.layers.xavier_initializer(
                    seed=self.random_seed)))
            
            self.biases_p.append(tf.get_variable(
                name=bias_key, shape=[d_out],
                initializer=tf.truncated_normal_initializer(
                    stddev=0.001, seed=self.random_seed)))
            
            # add summary stats
            tf.summary.histogram(weight_key, self.weights_p[-1])
            tf.summary.histogram(bias_key, self.biases_p[-1])

### Training/validation data, hyperparameters

Load the pre-processed training and validation data

In [32]:
unique_sid = list()
with open(os.path.join(pro_dir, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

n_items = len(unique_sid)
print(n_items)

75


In [33]:
def load_train_data(csv_file):
    tp = pd.read_csv(csv_file)
    n_users = tp['uid'].max() + 1

    rows, cols = tp['uid'], tp['sid']
    data = sparse.csr_matrix((np.ones_like(rows),
                             (rows, cols)), dtype='float64',
                             shape=(n_users, n_items))
    return data

In [34]:
train_data = load_train_data(os.path.join(pro_dir, 'train.csv'))

In [35]:
def load_tr_te_data(csv_file_tr, csv_file_te):
    tp_tr = pd.read_csv(csv_file_tr)
    tp_te = pd.read_csv(csv_file_te)

    start_idx = min(tp_tr['uid'].min(), tp_te['uid'].min())
    end_idx = max(tp_tr['uid'].max(), tp_te['uid'].max())

    rows_tr, cols_tr = tp_tr['uid'] - start_idx, tp_tr['sid']
    rows_te, cols_te = tp_te['uid'] - start_idx, tp_te['sid']

    data_tr = sparse.csr_matrix((np.ones_like(rows_tr),
                             (rows_tr, cols_tr)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    data_te = sparse.csr_matrix((np.ones_like(rows_te),
                             (rows_te, cols_te)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    return data_tr, data_te

In [36]:
vad_data_tr, vad_data_te = load_tr_te_data(os.path.join(pro_dir, 'validation_tr.csv'),
                                           os.path.join(pro_dir, 'validation_te.csv'))

In [37]:
print(train_data.shape)
print(vad_data_tr.shape, vad_data_te.shape)

(100, 75)
(100, 75) (100, 75)


Set up training hyperparameters

In [38]:
N = train_data.shape[0]
idxlist = list(range(N))

# training batch size
batch_size = 25 #coldstart=25 #MRM=500
batches_per_epoch = int(np.ceil(float(N) / batch_size))

N_vad = vad_data_tr.shape[0]
idxlist_vad = range(N_vad)

# validation batch size (since the entire validation set might not fit into GPU memory)
batch_size_vad = 50 #coldstart=25 #MRM=50 # 2000

# the total number of gradient updates for annealing
total_anneal_steps = 200000
# largest annealing parameter
anneal_cap = 0.2

Evaluate function: Normalized discounted cumulative gain (NDCG@k) and Recall@k

In [39]:
def NDCG_binary_at_k_batch(X_pred, heldout_batch, k=10): #coldstart=10 #MRM=100
    '''
    normalized discounted cumulative gain@k for binary relevance
    ASSUMPTIONS: all the 0's in heldout_data indicate 0 relevance
    '''
    batch_users = X_pred.shape[0] # 150
    idx_topk_part = bn.argpartition(-X_pred, k, axis=1)
    topk_part = X_pred[np.arange(batch_users)[:, np.newaxis],
                       idx_topk_part[:, :k]]
    idx_part = np.argsort(-topk_part, axis=1)
    # X_pred[np.arange(batch_users)[:, np.newaxis], idx_topk] is the sorted
    # topk predicted score
    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]
    # build the discount template
    tp = 1. / np.log2(np.arange(2, k + 2))

    DCG = (heldout_batch[np.arange(batch_users)[:, np.newaxis],
                         idx_topk].toarray() * tp).sum(axis=1)
    IDCG = np.array([(tp[:min(n, k)]).sum()
                     for n in heldout_batch.getnnz(axis=1)])
    return DCG / IDCG

In [40]:
def Recall_at_k_batch(X_pred, heldout_batch, k=5):
    batch_users = X_pred.shape[0]
    idx = bn.argpartition(-X_pred, k, axis=1)
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True

    X_true_binary = (heldout_batch > 0).toarray()
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(
        np.float32)
    
    recall = tmp / X_true_binary.sum(axis=1)
    rec = np.sum(tmp) / np.sum(X_true_binary.sum(axis=1))
    return recall, rec

In [41]:
def Prec_at_k_batch(X_pred, heldout_batch, k=5):
    batch_users = X_pred.shape[0]
    idx = bn.argpartition(-X_pred, k, axis=1)
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True

    X_true_binary = (heldout_batch > 0).toarray()
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(
        np.float32)
    
#     prec = tmp / np.minimum(k, X_pred_binary.sum(axis=1))
    prec = np.sum(tmp) / (batch_users * k)
    return prec

In [42]:
from sklearn.metrics import average_precision_score

def MAP(X_pred,heldout_batch):
    X_pred[X_pred == -np.inf] = -9999999999
    batch_users = X_pred.shape[0]
    X_true_binary = heldout_batch.toarray()
    
    total_prec = 0
    for u in range(batch_users):
        y_true = X_true_binary[u]
        y_scores = X_pred[u]
        total_prec += average_precision_score(y_true, y_scores)
        
    Map_value = total_prec/batch_users
    
    return Map_value

In [43]:
def F1_score(prec,rec):
    f1 = 2*((prec*rec)/(prec+rec))
    return f1

### Train a Multi-VAE^{PR}

For ML-20M dataset, we set both the generative function $f_\theta(\cdot)$ and the inference model $g_\phi(\cdot)$ to be 3-layer multilayer perceptron (MLP) with symmetrical architecture. 

The generative function is a [200 -> 600 -> n_items] MLP, which means the inference function is a [n_items -> 600 -> 200] MLP. Thus the overall architecture for the Multi-VAE^{PR} is [n_items -> 600 -> 200 -> 600 -> n_items].

In [44]:
p_dims = [200, 600, n_items]

In [45]:
tf.reset_default_graph()
vae = MultiVAE(p_dims, lam=0.0, random_seed=98765)

saver, logits_var, loss_var, train_op_var, merged_var = vae.build_graph()

ndcg_var = tf.Variable(0.0)
ndcg_dist_var = tf.placeholder(dtype=tf.float64, shape=None)
ndcg_summary = tf.summary.scalar('ndcg_at_k_validation', ndcg_var)
ndcg_dist_summary = tf.summary.histogram('ndcg_at_k_hist_validation', ndcg_dist_var)
merged_valid = tf.summary.merge([ndcg_summary, ndcg_dist_summary])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
INFO:tensorflow:Scale of 0 disables regularizer.
Instructions for updating:
Use tf.cast instead.


Set up logging and checkpoint directory

- Change all the logging directory and checkpoint directory to somewhere of your choice
- Monitor training progress using tensorflow by: `tensorboard --logdir=$log_dir`

In [46]:
arch_str = "I-%s-I" % ('-'.join([str(d) for d in vae.dims[1:-1]]))

In [47]:
log_dir = './volmount/log/ml-20m/VAE_anneal{}K_cap{:.0E}/{}'.format(
    int(total_anneal_steps/1000), anneal_cap, arch_str)

if os.path.exists(log_dir):
    shutil.rmtree(log_dir)

print("log directory: %s" % log_dir)
summary_writer = tf.summary.FileWriter(log_dir, graph=tf.get_default_graph())

log directory: ./volmount/log/ml-20m/VAE_anneal200K_cap2E-01/I-600-200-600-I


In [48]:
chkpt_dir = './volmount/chkpt/ml-20m/VAE_anneal{}K_cap{:.0E}/{}'.format(
    int(total_anneal_steps/1000), anneal_cap, arch_str)

if not os.path.isdir(chkpt_dir):
    os.makedirs(chkpt_dir) 
    
print("chkpt directory: %s" % chkpt_dir)

chkpt directory: ./volmount/chkpt/ml-20m/VAE_anneal200K_cap2E-01/I-600-200-600-I


In [49]:
n_epochs = 200

In [50]:
ndcgs_vad = []

with tf.Session() as sess:

    init = tf.global_variables_initializer()
    sess.run(init)

    best_ndcg = -np.inf

    update_count = 0.0
    
    for epoch in range(n_epochs):
        np.random.shuffle(idxlist)
        # train for one epoch
        for bnum, st_idx in enumerate(range(0, N, batch_size)):
            end_idx = min(st_idx + batch_size, N)
            X = train_data[idxlist[st_idx:end_idx]]
            
            if sparse.isspmatrix(X):
                X = X.toarray()
            X = X.astype('float32')           
            
            if total_anneal_steps > 0:
                anneal = min(anneal_cap, 1. * update_count / total_anneal_steps)
            else:
                anneal = anneal_cap
            
            feed_dict = {vae.input_ph: X, 
                         vae.keep_prob_ph: 0.5, 
                         vae.anneal_ph: anneal,
                         vae.is_training_ph: 1}        
            sess.run(train_op_var, feed_dict=feed_dict)

            if bnum % 100 == 0:
                summary_train = sess.run(merged_var, feed_dict=feed_dict)
                summary_writer.add_summary(summary_train, 
                                           global_step=epoch * batches_per_epoch + bnum) 
            
            update_count += 1
        
        # compute validation NDCG
        ndcg_dist = []
        for bnum, st_idx in enumerate(range(0, N_vad, batch_size_vad)):
            end_idx = min(st_idx + batch_size_vad, N_vad)
            X = vad_data_tr[idxlist_vad[st_idx:end_idx]]

            if sparse.isspmatrix(X):
                X = X.toarray()
            X = X.astype('float32')
        
            pred_val = sess.run(logits_var, feed_dict={vae.input_ph: X} )
            # exclude examples from training and validation (if any)
            pred_val[X.nonzero()] = -np.inf
            ndcg_dist.append(NDCG_binary_at_k_batch(pred_val, vad_data_te[idxlist_vad[st_idx:end_idx]]))
        
        ndcg_dist = np.concatenate(ndcg_dist)
        ndcg_ = ndcg_dist.mean()
        ndcgs_vad.append(ndcg_)
        merged_valid_val = sess.run(merged_valid, feed_dict={ndcg_var: ndcg_, ndcg_dist_var: ndcg_dist})
        summary_writer.add_summary(merged_valid_val, epoch)

        # update the best model (if necessary)
        if ndcg_ > best_ndcg:
            saver.save(sess, '{}/model'.format(chkpt_dir))
            best_ndcg = ndcg_

/home/tonylab/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


InvalidArgumentError: Nan in summary histogram for: ndcg_at_k_hist_validation
	 [[node ndcg_at_k_hist_validation (defined at <ipython-input-45-44b6ba079393>:9) ]]

Caused by op 'ndcg_at_k_hist_validation', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/tonylab/.local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/tonylab/.local/lib/python3.6/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/tonylab/.local/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  File "/home/tonylab/.local/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.6/asyncio/base_events.py", line 438,

In [ ]:
plt.figure(figsize=(12, 3))
plt.plot(ndcgs_vad)
plt.ylabel("Validation NDCG@10")
plt.xlabel("Epochs")
pass

### Load the test data and compute test metrics

In [ ]:
test_data_tr, test_data_te = load_tr_te_data(
    os.path.join(pro_dir, 'test_tr.csv'),
    os.path.join(pro_dir, 'test_te.csv'))

In [ ]:
N_test = test_data_tr.shape[0]
idxlist_test = range(N_test)

batch_size_test = 2000

In [ ]:
tf.reset_default_graph()
vae = MultiVAE(p_dims, lam=0.0)
saver, logits_var, _, _, _ = vae.build_graph()    

Load the best performing model on the validation set

In [ ]:
# chkpt_dir = '/volmount/chkpt/ml-20m/VAE_anneal{}K_cap{:.0E}/{}'.format(
#     int(total_anneal_steps/1000), anneal_cap, arch_str)
# print("chkpt directory: %s" % chkpt_dir)

In [ ]:
n10_list = []
r1_list, r5_list = [], []
# p1_list, p5_list = [], []

with tf.Session() as sess:
    saver.restore(sess, '{}/model'.format(chkpt_dir))

    for bnum, st_idx in enumerate(range(0, N_test, batch_size_test)):
        end_idx = min(st_idx + batch_size_test, N_test)
        X = test_data_tr[idxlist_test[st_idx:end_idx]]

        if sparse.isspmatrix(X):
            X = X.toarray()
        X = X.astype('float32')

        pred_val = sess.run(logits_var, feed_dict={vae.input_ph: X})
        # exclude examples from training and validation (if any)
        pred_val[X.nonzero()] = -np.inf
        
        n10_list.append(NDCG_binary_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=10))
        MAP = MAP(pred_val, test_data_te[idxlist_test[st_idx:end_idx]])

#         p1_list.append(Prec_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=1))
        p1 = Prec_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=1)
#         r1_list.append(Recall_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=1))
        recall, rec_1 = Recall_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=1)
        r1_list.append(recall)
        
#         p5_list.append(Prec_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=5))
        p5 = Prec_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=5)
#         r5_list.append(Recall_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=5))
        recall, rec_5 = Recall_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=5)
        r5_list.append(recall)

n10_list = np.concatenate(n10_list)

# p1_list = np.concatenate(p1_list)
r1_list = np.concatenate(r1_list)

# p5_list = np.concatenate(p5_list)
r5_list = np.concatenate(r5_list)

In [ ]:
# print("Test NDCG@10=%f (%f)" % (np.mean(n10_list), np.std(n10_list) / np.sqrt(len(n10_list))))
print("Test NDCG@10={}".format(np.mean(n10_list)))
print("Test MAP={}".format(MAP))
# print("Test Prec@1=%f (%f)" % (np.mean(p1_list), np.std(p1_list) / np.sqrt(len(p1_list))))
print("Test Prec@1={}".format(p1))
# print("Test Recall@1=%f (%f)" % (np.mean(r1_list), np.std(r1_list) / np.sqrt(len(r1_list))))
# print("Test Recall@1={}".format(np.mean(r1_list)))
# print('F1@1', F1_score(p1, np.mean(r1_list)))

print('Recall:', rec_1)
print('F1@1', F1_score(p1, rec_1))

# print("Test Prec@5=%f (%f)" % (np.mean(p5_list), np.std(p5_list) / np.sqrt(len(p5_list))))
print("Test Prec@5={}".format(p5))
# print("Test Recall@5=%f (%f)" % (np.mean(r5_list), np.std(r5_list) / np.sqrt(len(r5_list))))
# print("Test Recall@5={}".format(np.mean(r5_list)))
# print('F1@1', F1_score(p5, np.mean(r5_list)))
print('Recall:', rec_5)
print('F1@5', F1_score(p5, rec_5))

# Metrics

### Train a Multi-DAE

The generative function is a [200 -> n_items] MLP, thus the overall architecture for the Multi-DAE is [n_items -> 200 -> n_items]. We find this architecture achieves better validation NDCG@100 than the [n_items -> 600 -> 200 -> 600 -> n_items] architecture as used in Multi-VAE^{PR}.

In [ ]:
p_dims = [200, n_items]

In [ ]:
tf.reset_default_graph()
dae = MultiDAE(p_dims, lam=0.01 / batch_size, random_seed=98765)

saver, logits_var, loss_var, train_op_var, merged_var = dae.build_graph()

ndcg_var = tf.Variable(0.0)
ndcg_dist_var = tf.placeholder(dtype=tf.float64, shape=None)
ndcg_summary = tf.summary.scalar('ndcg_at_k_validation', ndcg_var)
ndcg_dist_summary = tf.summary.histogram('ndcg_at_k_hist_validation', ndcg_dist_var)
merged_valid = tf.summary.merge([ndcg_summary, ndcg_dist_summary])

Set up logging and checkpoint directory

In [ ]:
arch_str = "I-%s-I" % ('-'.join([str(d) for d in dae.dims[1:-1]]))

In [ ]:
log_dir = './volmount/log/ml-20m/DAE/{}'.format(arch_str)

if os.path.exists(log_dir):
    shutil.rmtree(log_dir)

print("log directory: %s" % log_dir)
summary_writer = tf.summary.FileWriter(log_dir, graph=tf.get_default_graph())

In [ ]:
chkpt_dir = './volmount/chkpt/ml-20m/DAE/{}'.format(arch_str)

if not os.path.isdir(chkpt_dir):
    os.makedirs(chkpt_dir) 
    
print("chkpt directory: %s" % chkpt_dir)

In [ ]:
n_epochs = 200

In [ ]:
ndcgs_vad = []

with tf.Session() as sess:

    init = tf.global_variables_initializer()
    sess.run(init)

    best_ndcg = -np.inf
    
    for epoch in range(n_epochs):
        np.random.shuffle(idxlist)
        # train for one epoch
        for bnum, st_idx in enumerate(range(0, N, batch_size)):
            end_idx = min(st_idx + batch_size, N)
            X = train_data[idxlist[st_idx:end_idx]]
            
            if sparse.isspmatrix(X):
                X = X.toarray()
            X = X.astype('float32')           
            
            feed_dict = {dae.input_ph: X, 
                         dae.keep_prob_ph: 0.5}        
            sess.run(train_op_var, feed_dict=feed_dict)

            if bnum % 100 == 0:
                summary_train = sess.run(merged_var, feed_dict=feed_dict)
                summary_writer.add_summary(summary_train, global_step=epoch * batches_per_epoch + bnum) 
                    
        # compute validation NDCG
        ndcg_dist = []
        for bnum, st_idx in enumerate(range(0, N_vad, batch_size_vad)):
            end_idx = min(st_idx + batch_size_vad, N_vad)
            X = vad_data_tr[idxlist_vad[st_idx:end_idx]]

            if sparse.isspmatrix(X):
                X = X.toarray()
            X = X.astype('float32')
        
            pred_val = sess.run(logits_var, feed_dict={dae.input_ph: X} )
            # exclude examples from training and validation (if any)
            pred_val[X.nonzero()] = -np.inf
            ndcg_dist.append(NDCG_binary_at_k_batch(pred_val, vad_data_te[idxlist_vad[st_idx:end_idx]]))
        
        ndcg_dist = np.concatenate(ndcg_dist)
        ndcg_ = ndcg_dist.mean()
        ndcgs_vad.append(ndcg_)
        merged_valid_val = sess.run(merged_valid, feed_dict={ndcg_var: ndcg_, ndcg_dist_var: ndcg_dist})
        summary_writer.add_summary(merged_valid_val, epoch)

        # update the best model (if necessary)
        if ndcg_ > best_ndcg:
            saver.save(sess, '{}/model'.format(chkpt_dir))
            best_ndcg = ndcg_

In [ ]:
plt.figure(figsize=(12, 3))
plt.plot(ndcgs_vad)
plt.ylabel("Validation NDCG@10")
plt.xlabel("Epochs")
pass

### Compute test metrics

In [ ]:
tf.reset_default_graph()
dae = MultiDAE(p_dims, lam=0.01 / batch_size)
saver, logits_var, _, _, _ = dae.build_graph()    

Load the best performing model on the validation set

In [ ]:
# chkpt_dir = '/volmount/chkpt/ml-20m/DAE/{}'.format(arch_str)
# print("chkpt directory: %s" % chkpt_dir)

In [ ]:
n100_list, r20_list, r50_list = [], [], []

with tf.Session() as sess:    
    saver.restore(sess, '{}/model'.format(chkpt_dir))
    
    for bnum, st_idx in enumerate(range(0, N_test, batch_size_test)):
        end_idx = min(st_idx + batch_size_test, N_test)
        X = test_data_tr[idxlist_test[st_idx:end_idx]]

        if sparse.isspmatrix(X):
            X = X.toarray()
        X = X.astype('float32')

        pred_val = sess.run(logits_var, feed_dict={dae.input_ph: X})
        # exclude examples from training and validation (if any)
        pred_val[X.nonzero()] = -np.inf
        n100_list.append(NDCG_binary_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=10))
        r20_list.append(Recall_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=1))
        r50_list.append(Recall_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=5))

n100_list = np.concatenate(n100_list)
r20_list = np.concatenate(r20_list)
r50_list = np.concatenate(r50_list)

In [ ]:
print("Test NDCG@10=%.5f (%.5f)" % (np.mean(n100_list), np.std(n100_list) / np.sqrt(len(n100_list))))
print("Test Recall@1=%.5f (%.5f)" % (np.mean(r20_list), np.std(r20_list) / np.sqrt(len(r20_list))))
print("Test Recall@5=%.5f (%.5f)" % (np.mean(r50_list), np.std(r50_list) / np.sqrt(len(r50_list))))